In [1]:
import pandas as pd

In [3]:
v1_res = pd.read_csv('results/BTC_trades.csv')
v2_res = pd.read_csv('results/BTC_trades_backtestingpy.csv')


In [5]:
v1_res.pnl.sum()

np.float64(315044.9226230075)

In [10]:
v2_res.PnL.sum()


np.float64(307829.89999999997)